In [1]:
!pip install biopython


Defaulting to user installation because normal site-packages is not writeable


In [1]:
import pandas as pd
import time
from Bio import Entrez

def fetch_snp_info(rsid):
    try:
        Entrez.email = "ilcardenas26@gmail.com"  # Replace with your email
        handle = Entrez.efetch(db="snp", id=rsid, rettype="docset", retmode="text")
        time.sleep(3)  # Be mindful of API rate limits
        return handle.read()
    except Exception as e:
        print(f"Error fetching data for {rsid}: {e}")
        return None

# Read your file with rsIDs
file_path = "AncestryDNA.txt"
df = pd.read_csv(file_path, delimiter="\t")

# Ensure 'snp_info' column exists
if 'snp_info' not in df.columns:
    df['snp_info'] = None

# Fetch and save data row by row
output_file = "snp_info_output.csv"


In [5]:
for index, row in df.iterrows():
    if pd.isna(row['snp_info']):  # Skip already fetched rows
        for row2 in df.iterrows():
            if row2[4].isna(): 
                rsid = row['rsid']
                print(f"Fetching data for {rsid}...")
                snp_info = fetch_snp_info(rsid)
                df.at[index, 'snp_info'] = snp_info
                # Save after each fetch
                df.to_csv(output_file, index=False)
                print(f"Saved progress after {rsid}.")

IndexError: tuple index out of range

In [6]:
import csv
import time
from Bio import Entrez

def fetch_snp_info(rsid):
    try:
        Entrez.email = "ilcardenas26@gmail.com"
        handle = Entrez.efetch(db="snp", id=rsid, rettype="docset", retmode="text")
        time.sleep(3)
        return handle.read()
    except Exception as e:
        print(f"Error fetching data for {rsid}: {e}")
        return None

# Input and output files
input_file = "AncestryDNA.txt"
output_file = "snp_info_output.csv"

# Process line by line
with open(input_file, "r") as infile, open(output_file, "a", newline="") as outfile:
    reader = csv.DictReader(infile, delimiter="\t")
    fieldnames = reader.fieldnames + ["snp_info"]
    writer = csv.DictWriter(outfile, fieldnames=fieldnames)

    # Write header if output file is empty
    if outfile.tell() == 0:
        writer.writeheader()

    for row in reader:
        if "snp_info" not in row or not row["snp_info"]:  # Skip if already fetched
            print(f"Fetching data for {row['rsid']}...")
            row["snp_info"] = fetch_snp_info(row["rsid"])
            writer.writerow(row)
            print(f"Saved {row['rsid']} to file.")

Fetching data for rs3131972...
Saved rs3131972 to file.
Fetching data for rs114525117...
Saved rs114525117 to file.
Fetching data for rs4040617...
Saved rs4040617 to file.
Fetching data for rs141175086...
Saved rs141175086 to file.
Fetching data for rs115093905...
Saved rs115093905 to file.
Fetching data for rs11240777...
Saved rs11240777 to file.
Fetching data for rs6681049...
Saved rs6681049 to file.
Fetching data for rs4422948...
Saved rs4422948 to file.
Fetching data for rs57494724...
Saved rs57494724 to file.
Fetching data for rs4475691...
Saved rs4475691 to file.
Fetching data for rs6657440...
Saved rs6657440 to file.
Fetching data for rs4970461...
Saved rs4970461 to file.
Fetching data for rs7537756...
Saved rs7537756 to file.
Fetching data for rs13302982...
Saved rs13302982 to file.
Fetching data for rs2880024...
Saved rs2880024 to file.
Fetching data for rs74047407...
Saved rs74047407 to file.
Fetching data for rs1110052...
Saved rs1110052 to file.
Fetching data for rs7523549.

KeyboardInterrupt: 

In [5]:
import sqlite3
import pandas as pd
import time
from Bio import Entrez
import multiprocessing
from multiprocessing import Pool, cpu_count
from tqdm import tqdm


def fetch_snp_info(rsid):
    """
    Fetch SNP info using NCBI Entrez API for a given rsID.
    """
    try:
        Entrez.email = "ilcardenas26@gmail.com"
        # print(f"Processing rsID: {rsid}")  # Log the rsID being processed
        handle = Entrez.efetch(db="snp", id=rsid, rettype="docset", retmode="text")
        time.sleep(1)  # Respect API rate limits
        return rsid, handle.read()

    except Exception as e:
        print(f"Error fetching data for {rsid}: {e}")
        if "Error 400" in e:
            return rsid, "No data found in SNP database."
        else: 
            return rsid, None


def process_chunk(rsid_chunk):
    """
    Process a chunk of rsIDs by fetching SNP info.
    """
    results = []
    for rsid in rsid_chunk:
        results.append(fetch_snp_info(rsid))
    return results


def update_database(db_file, results):
    """
    Update the SQLite database with the fetched SNP info.
    """
    conn = sqlite3.connect(db_file)
    print("Connected to DB")
    cursor = conn.cursor()
    print("created cursor")
    for rsid, snp_info in results:
        print(rsid, snp_info)
        cursor.execute("UPDATE snp_data SET snp_info = ? WHERE rsid = ?", (snp_info, rsid))
        print('Item updated')
    conn.commit()
    conn.close()


def main():
    # Input and database setup
    input_file = "AncestryDNA.txt"
    db_file = "snp_data.db"

    # Create SQLite database and load data if not already present
    # conn = sqlite3.connect(db_file)
    # df = pd.read_csv(input_file, delimiter="\t")
    # df.to_sql("snp_data", conn, if_exists="replace", index=False)
    
    # # Add the 'snp_info' column if it doesn't exist
    # if 'snp_info' not in df.columns:
    #     df['snp_info'] = None

    # # Save to the database
    # df.to_sql("snp_data", conn, if_exists="replace", index=False)
    # conn.close()
    

    # Fetch all rsIDs with missing SNP info
    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()

    # cursor.execute(("CREATE INDEX indexing ON snp_data (rsid);"))
    cursor.execute("SELECT rsid FROM snp_data WHERE snp_info IS NULL")
    rsid_list = [row[0] for row in cursor.fetchall()]
    print("RSID List created")
    print (rsid_list[:5])
    conn.close()

    # Divide rsIDs into chunks for multiprocessing
    
    print("Starting chunking")
    num_processes = min(cpu_count(), len(rsid_list))  # Use as many processes as available cores
    # chunk_size = len(rsid_list) // num_processes + 1
    chunk_size = 10
   
    rsid_chunks = [rsid_list[i:i + chunk_size] for i in range(0, len(rsid_list), chunk_size)]
    print("Chunks: ", len(rsid_chunks))

    print("Finished Chuncking")
    return rsid_chunks, rsid_list, num_processes
if __name__ == "__main__":
    rsid_chuncks, rsid_list, num_procesess = main()  

RSID List created
['rs2274328', 'rs2274330', 'rs17032907', 'rs3765968', 'rs3765966']
Starting chunking
Chunks:  67500
Finished Chuncking


In [ ]:
rsid_chuncks, rsid_list, num_procesess 

In [6]:
db_file = "snp_data.db"
def printtest(id):
        print(id)
        
conn = sqlite3.connect(db_file)
print("Connected to DB")
cursor = conn.cursor()
print("created cursor")


             
with Pool(12) as pool:
        for segment in rsid_chuncks:
                for i in tqdm(range(0,len(rsid_chuncks)), total=len(rsid_chuncks), desc="Progress"):              
                        for item in segment:
                                rsid, snp_info = fetch_snp_info(item)
                                # item = fetch_snp_info(row["rsid"])
                                cursor.execute("UPDATE snp_data SET snp_info = ? WHERE rsid = ?", (snp_info, rsid))
                                conn.commit()
                               
conn.close()              

    # # Save the final updated data to CSV
    # conn = sqlite3.connect(db_file)
    # df_updated = pd.read_sql("SELECT * FROM snp_data", conn)
    # # df_updated.to_csv("snp_info_output.csv", index=False)
    # conn.close()

print("Processing complete. Data saved to snp_info_output.csv.")

Connected to DB
created cursor


Progress:   0%|          | 1/67500 [00:24<453:36:19, 24.19s/it]

Error fetching data for rs2274328: HTTP Error 400: Bad Request


Progress:   0%|          | 17/67500 [06:27<428:21:56, 22.85s/it]

Error fetching data for rs2274330: HTTP Error 400: Bad Request


...

Progress:   1%|          | 789/67500 [4:57:15<401:36:40, 21.67s/it]

Error fetching data for rs3765968: HTTP Error 400: Bad Request


Progress:   1%|          | 801/67500 [5:02:01<419:09:01, 22.62s/it]

Error fetching data for rs17389460: IncompleteRead(3520 bytes read)


TypeError: argument of type 'IncompleteRead' is not iterable